In [30]:
from sklearn.datasets import make_classification
X, Y = make_classification(
  n_samples=100, n_features=4, n_redundant=0,
  n_informative=3,  n_clusters_per_class=2, n_classes=3
)#This is to generate the data

In [31]:
#print(X,Y)#so X is the data and Y is the label.
print(len(X),len(Y))

100 100


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
  X, Y, test_size=0.33, random_state=42)

In [33]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self, X_train, y_train):
    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
        self.X = torch.from_numpy(X_train.astype(np.float32))
    # need to convert float64 to Long else 
    # will get the following error
    # RuntimeError: expected scalar type Long but found Float
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]
  
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    def __len__(self):
        return self.len

In [34]:
traindata = Data(X_train, Y_train)

In [53]:
print(traindata[25:34])


(tensor([[ 2.4495, -0.7271, -0.6518, -1.2514],
        [ 0.5928, -0.8193, -1.6907, -1.2532],
        [ 2.7793, -2.3289,  1.2482, -0.3264],
        [-1.0350, -0.0695,  0.4478,  0.6540],
        [-0.5648,  1.1119,  0.4620, -1.6004],
        [ 2.4732, -0.2942, -0.7508, -1.9410],
        [ 0.3064,  1.8329, -0.9883,  1.6277],
        [ 0.2584,  2.3067, -0.1299,  1.7404],
        [-0.1217,  2.2558,  0.5328, -2.6542]]), tensor([0, 0, 1, 2, 2, 0, 1, 1, 0]))


In [36]:
batch_size = 4
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True)#num_workers is for parallel loading, caused problem
#RuntimeError: DataLoader worker (pid(s) 45243, 45244) exited unexpectedly

In [37]:
import torch.nn as nn
# number of features (len of X cols)
input_dim = 4
# number of hidden layers
hidden_layers = 25
# number of classes (unique of y)
output_dim = 3
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_layers)
        self.linear2 = nn.Linear(hidden_layers, output_dim)
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [38]:
clf = Network()
#print(clf.parameters)

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

In [47]:
epochs = 2
for epoch in range(epochs):
    running_loss = 0.0
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    loss = criterion(outputs, labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
  # display statistics
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[2,     1] loss: 0.00048
[2,     2] loss: 0.00099
[2,     3] loss: 0.00147
[2,     4] loss: 0.00196
[2,     5] loss: 0.00256
[2,     6] loss: 0.00308
[2,     7] loss: 0.00355
[2,     8] loss: 0.00403
[2,     9] loss: 0.00455
[2,    10] loss: 0.00509
[2,    11] loss: 0.00558
[2,    12] loss: 0.00612
[2,    13] loss: 0.00662
[2,    14] loss: 0.00716
[2,    15] loss: 0.00763
[2,    16] loss: 0.00814
[2,    17] loss: 0.00865


In [48]:
# save the trained model
PATH = './mymodel.pth'
torch.save(clf.state_dict(), PATH)

In [49]:
clf = Network()
clf.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [50]:
testdata = Data(X_test, Y_test)
testloader = DataLoader(testdata, batch_size=batch_size, 
                        shuffle=True)
dataiter = iter(testloader)
inputs, labels = next(dataiter)
print(inputs)
print(labels)

tensor([[ 0.5429, -0.5266, -0.8194, -1.9244],
        [ 1.6707, -0.6434, -0.2873, -0.0788],
        [ 1.5735,  3.0698,  0.1109, -3.2220],
        [-0.4222,  1.3163,  1.4754, -2.0970]])
tensor([0, 1, 0, 0])


In [51]:
outputs = clf(inputs)
__, predicted = torch.max(outputs, 1)
print(predicted)
'''
# Output
tensor([0, 0, 1, 0])
'''

tensor([2, 1, 1, 2])


'\n# Output\ntensor([0, 0, 1, 0])\n'

In [52]:
correct, total = 0, 0
# no need to calculate gradients during inference
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
    # calculate output by running through the network
        outputs = clf(inputs)
    # get the predictions
        __, predicted = torch.max(outputs.data, 1)
    # update results
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the {len(testdata)} test data: {100 * correct // total} %')
'''
# Output
Accuracy of the network on the 33 test data: 75 %
'''

Accuracy of the network on the 33 test data: 51 %


'\n# Output\nAccuracy of the network on the 33 test data: 75 %\n'